In [19]:
import re
import csv
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import astrometry  # pip install astrometry
from scipy.spatial import cKDTree  # pip install scipy

from astropy.io import fits
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.stats import sigma_clipped_stats

from photutils.detection import DAOStarFinder
from photutils.centroids import centroid_com, centroid_2dg
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry


# =============================================================================
# ============================== CONFIG VARS ==================================
# =============================================================================

VIDEO_PATH = "/Users/gustavoschwarz/Downloads/2025-09-01-014738-Uranus-RAW.avi"
OUTPUT_SUFFIX = "Uranus1"

# ---- paste your Seestar log block here ----
META_TEXT = r"""
RA [ICRS]: 04 12 36.2116, Dec [ICRS]: +26 32 56.331

[Seestar S50]
Bin = 1
Capture Area Size = 1080 * 1920
Colour Format = RAW8
Flip = None
StartX = 0
StartY = 0
Temperature = 31.4375 C
Bayer = GR
White Balance (B) = 147
White Balance (R) = 132
DATE-OBS = 2025-09-01T04:47:39.216348
SITELONG = -48.462500
SITELAT = -7.791390
OBJECT = Uranus
Duration=520 Sec
""".strip()

# ---- USER MUST SET THESE (per your rule) ----
EXPOSURE_TIME = 0.065   # seconds (user-provided)
READOUT_GAP   = 0.002   # seconds (fixed gap)
DT_FRAME      = EXPOSURE_TIME + READOUT_GAP  # per-frame step in seconds

# If True: DATE-OBS from log is MID-exposure of frame0.
LOG_TIME_IS_MID_EXPOSURE = True

# ----------------- Local astrometry (Python) -----------------
ASTROMETRY_CACHE_DIR = "astrometry_cache"
ASTROMETRY_SERIES = "series_4200"
ASTROMETRY_SCALES = {5,6,7,8}

LOWER_ARCSEC_PER_PIXEL = 2.33
UPPER_ARCSEC_PER_PIXEL = 2.42
SEARCH_RADIUS_DEG = 1

SIP_ORDER = 0
MAX_LOGODDS_LIST_LEN = 15

# ----------------- Star detection -----------------
DAO_FWHM = 5.0
DAO_SIGMA = 3.5
MIN_FLUX = 1.09
REFINE_METHOD = "com"   # "gaussian" or "com" or "none"
CUTOUT_SIZE = 20

# detect stars every N frames (1 = every frame; use 2/3/5 if too slow)
DETECT_EVERY_N = 1

# ----------------- Matching / transform estimation -----------------
MATCH_MAX_DIST_PX = 120.0
MIN_MATCHES = 10

RANSAC_REPROJ_THRESH = 4.0
RANSAC_CONFIDENCE = 0.995

# ----------------- Differential photometry -----------------
AP_RADIUS = 5.0
ANN_IN = 8.0
ANN_OUT = 14.0

N_COMP_STARS = 5
COMP_EXCLUDE_RADIUS_PX = 30.0
COMP_EDGE_MARGIN_PX = 30

# ----------------- Outputs -----------------
OUTDIR = Path(f"out_{OUTPUT_SUFFIX}")
OUTDIR.mkdir(exist_ok=True)

# NOTE: salvar cube FITS exigiria reabrir e escrever incremental (não implementado aqui)
SAVE_CUBE_FITS = False
SAVE_INDIVIDUAL_FITS = False

CSV_FILE = OUTDIR / f"lightcurve_{OUTPUT_SUFFIX}.csv"

PLOT_FLUXREL = OUTDIR / f"plot_fluxrel_{OUTPUT_SUFFIX}.png"
PLOT_DMAG    = OUTDIR / f"plot_dmag_{OUTPUT_SUFFIX}.png"
PLOT_NORM3   = OUTDIR / f"plot_norm_target_ref_bkg_{OUTPUT_SUFFIX}.png"

# ----------------- Full-frame diagnostics -----------------
MAKE_DIAGNOSTIC_PLOTS = True
N_RANDOM_FRAMES = 10
RANDOM_SEED = 123

OVERLAY_DETECTIONS = True
MAX_DETECTIONS_TO_DRAW = 800
DETECTION_MARKER_SIZE = 28

# ---- Astrometry using stacked early frames ----
ASTROMETRY_STACK_N = 5          # 5–10 costuma ser ótimo
ASTROMETRY_STACK_METHOD = "sum" # "sum" | "mean" | "median"

# =============================================================================
# ============================== HELPERS ======================================
# =============================================================================

def parse_meta_block(txt: str) -> dict:
    meta = {}

    m = re.search(
        r"RA\s*\[ICRS\]\s*:\s*([0-9\s\.]+)\s*,\s*Dec\s*\[ICRS\]\s*:\s*([-\+0-9\s\.]+)",
        txt
    )
    if m:
        ra_s = " ".join(m.group(1).split())
        dec_s = " ".join(m.group(2).split())
        c = SkyCoord(ra_s, dec_s, unit=(u.hourangle, u.deg), frame="icrs")
        meta["TARGET_RA_DEG"] = float(c.ra.deg)
        meta["TARGET_DEC_DEG"] = float(c.dec.deg)

    for k in ["DATE-OBS", "SITELONG", "SITELAT", "OBJECT", "Bayer", "Colour Format", "Flip"]:
        mm = re.search(rf"^{re.escape(k)}\s*=\s*(.+)\s*$", txt, flags=re.MULTILINE)
        if mm:
            meta[k] = mm.group(1).strip()

    if "DATE-OBS" in meta:
        meta["DATEOBS_TIME"] = Time(meta["DATE-OBS"], format="isot", scale="utc")
    else:
        meta["DATEOBS_TIME"] = Time.now()

    return meta


def write_common_header(hdr, meta, t0_start, t0_mid):
    hdr["EXPTIME"] = (float(EXPOSURE_TIME), "Exposure time per frame [s] (user)")
    hdr["RDGAP"]   = (float(READOUT_GAP),   "Readout gap between frames [s]")
    hdr["DTFRAME"] = (float(DT_FRAME),      "Frame step = EXPTIME + RDGAP [s]")
    hdr["DATE-OBS"] = (t0_start.isot, "Start time of frame 0 (log - EXPTIME/2)")
    hdr["DATE-MID"] = (t0_mid.isot,   "Log timestamp (assumed mid-exposure frame0)")

    if "OBJECT" in meta:
        hdr["OBJECT"] = (meta["OBJECT"], "Object name")
    if "SITELAT" in meta:
        try:
            hdr["SITELAT"] = (float(meta["SITELAT"]), "Site latitude [deg]")
        except Exception:
            hdr["SITELAT"] = (meta["SITELAT"], "Site latitude")
    if "SITELONG" in meta:
        try:
            hdr["SITELONG"] = (float(meta["SITELONG"]), "Site longitude [deg]")
        except Exception:
            hdr["SITELONG"] = (meta["SITELONG"], "Site longitude")

def read_and_stack_first_frames(cap, n_stack=10, method="mean"):
    """
    Read first n_stack frames from current cap position (assumes position at 0 or desired start),
    return stacked grayscale float32 image.
    """
    frames = []
    n_stack = int(n_stack)

    for _ in range(min(n_stack, ASTROMETRY_STACK_MAX_FRAMES)):
        ret, frame_bgr = cap.read()
        if not ret:
            break
        g = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY).astype(np.float32)
        frames.append(g)

    if len(frames) == 0:
        raise RuntimeError("Could not read frames to stack for astrometry.")

    cube = np.stack(frames, axis=0)  # shape (n, H, W)

    if method == "sum":
        img = np.sum(cube, axis=0)
    elif method == "median":
        img = np.median(cube, axis=0)
    else:  # "mean"
        img = np.mean(cube, axis=0)

    # normalize to something DAOStarFinder likes (optional but helps):
    # keep as float32; no need to rescale to uint16
    return img.astype(np.float32), len(frames)

def get_stars_positions(img: np.ndarray):
    mean, med, std = sigma_clipped_stats(img, sigma=3.0)
    daofind = DAOStarFinder(fwhm=DAO_FWHM, threshold=DAO_SIGMA * std)
    sources = daofind(img.astype(np.float32) - med)
    if sources is None or len(sources) == 0:
        return None

    if "flux" in sources.colnames:
        sources = sources[sources["flux"] > MIN_FLUX]
    if sources is None or len(sources) == 0:
        return None

    if REFINE_METHOD.lower() in ("gaussian", "com"):
        refined_x = []
        refined_y = []
        half = CUTOUT_SIZE // 2
        H, W = img.shape

        for star in sources:
            x, y = float(star["xcentroid"]), float(star["ycentroid"])
            x0 = max(0, int(round(x)) - half)
            x1 = min(W, int(round(x)) + half + 1)
            y0 = max(0, int(round(y)) - half)
            y1 = min(H, int(round(y)) + half + 1)

            cut = img[y0:y1, x0:x1].astype(np.float32)
            if cut.size < 9:
                refined_x.append(x)
                refined_y.append(y)
                continue

            _, med2, _ = sigma_clipped_stats(cut, sigma=3.0)
            cut2 = cut - med2
            cut2[cut2 < 0] = 0

            if REFINE_METHOD.lower() == "com":
                cx, cy = centroid_com(cut2)
            else:
                cx, cy = centroid_2dg(cut2)

            refined_x.append(x0 + float(cx))
            refined_y.append(y0 + float(cy))

        sources["xcentroid"] = refined_x
        sources["ycentroid"] = refined_y

    return sources


def solve_astrometry_local(sources, init_ra_deg, init_dec_deg):
    stars = [[float(s["xcentroid"]), float(s["ycentroid"])] for s in sources]

    print(f"[info] Preparing astrometry.net solver with {len(stars)} stars...")
    series_obj = getattr(astrometry, ASTROMETRY_SERIES)
    index_files = series_obj.index_files(
        cache_directory=ASTROMETRY_CACHE_DIR,
        scales=ASTROMETRY_SCALES,
    )

    solver = astrometry.Solver(index_files)

    solution = solver.solve(
        stars=stars,
        size_hint=astrometry.SizeHint(
            lower_arcsec_per_pixel=LOWER_ARCSEC_PER_PIXEL,
            upper_arcsec_per_pixel=UPPER_ARCSEC_PER_PIXEL,
        ),
        position_hint=astrometry.PositionHint(
            ra_deg=float(init_ra_deg),
            dec_deg=float(init_dec_deg),
            radius_deg=float(SEARCH_RADIUS_DEG),
        ),
        solution_parameters=astrometry.SolutionParameters(
            sip_order=SIP_ORDER,
            tune_up_logodds_threshold=None,
            logodds_callback=lambda logodds_list: (
                astrometry.Action.STOP
                if len(logodds_list) >= MAX_LOGODDS_LIST_LEN
                else astrometry.Action.CONTINUE
            ),
        ),
    )

    match = None
    if hasattr(solution, "has_match") and solution.has_match():
        if hasattr(solution, "best_match"):
            match = solution.best_match()
        elif hasattr(solution, "matches") and solution.matches:
            match = solution.matches[0]
    if match is None and hasattr(solution, "matches") and solution.matches:
        match = solution.matches[0]

    if match is None:
        raise RuntimeError("Astrometry failed: no match. Not enough stars or wrong index/scale hints.")

    if hasattr(match, "astropy_wcs"):
        wcs = match.astropy_wcs()
        wcs_header = wcs.to_header()
        return wcs_header, wcs

    if hasattr(match, "wcs_fields"):
        wcs_header = match.wcs_fields
        wcs = WCS(wcs_header)
        return wcs_header, wcs

    raise RuntimeError("Matched solution but couldn't extract WCS.")


def pick_comp_stars_from_sources(sources, x_t, y_t, shape_hw):
    H, W = shape_hw
    if "flux" in sources.colnames:
        sources = sources[np.argsort(sources["flux"])[::-1]]

    comps = []
    for s in sources:
        x = float(s["xcentroid"])
        y = float(s["ycentroid"])

        if x < COMP_EDGE_MARGIN_PX or x > (W - COMP_EDGE_MARGIN_PX):
            continue
        if y < COMP_EDGE_MARGIN_PX or y > (H - COMP_EDGE_MARGIN_PX):
            continue
        if (x - x_t) ** 2 + (y - y_t) ** 2 < COMP_EXCLUDE_RADIUS_PX ** 2:
            continue

        comps.append((x, y))
        if len(comps) >= N_COMP_STARS:
            break

    if len(comps) == 0:
        raise RuntimeError("Could not pick comparison stars from detections.")
    return comps


def aperture_flux_and_bkg(frame, positions_xy):
    aper = CircularAperture(positions_xy, r=AP_RADIUS)
    ann  = CircularAnnulus(positions_xy, r_in=ANN_IN, r_out=ANN_OUT)

    phot_aper = aperture_photometry(frame, aper)
    phot_ann  = aperture_photometry(frame, ann)

    aper_sum = np.array(phot_aper["aperture_sum"], dtype=np.float64)
    ann_sum  = np.array(phot_ann["aperture_sum"], dtype=np.float64)

    bkg_per_pix = ann_sum / ann.area
    bkg_in_aper = bkg_per_pix * aper.area
    return aper_sum - bkg_in_aper, bkg_per_pix


def stretch_limits(img, p_low=0.1, p_high=99.5):
    vmin = np.nanpercentile(img, p_low)
    vmax = np.nanpercentile(img, p_high)
    if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin >= vmax:
        vmin = float(np.nanmin(img))
        vmax = float(np.nanmax(img))
    return vmin, vmax


def apply_affine(A2x3, xy):
    xy = np.asarray(xy, dtype=np.float32)
    ones = np.ones((xy.shape[0], 1), dtype=np.float32)
    xyh = np.hstack([xy, ones])
    out = (A2x3 @ xyh.T).T
    return out


def match_mutual_nn_by_radius(ref_xy, cur_xy, max_dist, min_matches):
    if ref_xy.shape[0] < min_matches or cur_xy.shape[0] < min_matches:
        return None, None

    tree_cur = cKDTree(cur_xy)
    d_rc, idx_rc = tree_cur.query(ref_xy, distance_upper_bound=max_dist)
    good1 = np.isfinite(d_rc)
    if np.count_nonzero(good1) < min_matches:
        return None, None

    tree_ref = cKDTree(ref_xy)
    d_cr, idx_cr = tree_ref.query(cur_xy, distance_upper_bound=max_dist)
    good2 = np.isfinite(d_cr)

    ref_idx = np.where(good1)[0]
    cur_idx = idx_rc[good1]

    mutual = []
    for ri, cj in zip(ref_idx, cur_idx):
        if cj < 0 or cj >= len(cur_xy):
            mutual.append(False)
            continue
        mutual.append(bool(good2[cj] and idx_cr[cj] == ri))

    mutual = np.array(mutual, dtype=bool)
    if np.count_nonzero(mutual) < min_matches:
        return None, None

    ref_m = ref_xy[ref_idx[mutual]].astype(np.float32)
    cur_m = cur_xy[cur_idx[mutual]].astype(np.float32)
    return ref_m, cur_m


def estimate_similarity_ransac(ref_m, cur_m, min_matches):
    A, inliers = cv2.estimateAffinePartial2D(
        ref_m, cur_m,
        method=cv2.RANSAC,
        ransacReprojThreshold=RANSAC_REPROJ_THRESH,
        confidence=RANSAC_CONFIDENCE,
        refineIters=15,
    )
    if A is None or inliers is None:
        return None, 0
    nin = int(np.sum(inliers))
    if nin < min_matches:
        return None, nin
    return A.astype(np.float32), nin

def save_frame_percentiles(img, outpath, title="", p_low=0.1, p_high=99.5):
    vmin = np.nanpercentile(img, p_low)
    vmax = np.nanpercentile(img, p_high)
    if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin >= vmax:
        vmin, vmax = float(np.nanmin(img)), float(np.nanmax(img))

    plt.figure(figsize=(10, 8))
    plt.imshow(img, origin="lower", vmin=vmin, vmax=vmax)
    plt.colorbar(label="ADU (scaled)")
    plt.title(f"{title}\npercentiles: {p_low}%={vmin:.3g}, {p_high}%={vmax:.3g}")
    plt.xlabel("x [pix]")
    plt.ylabel("y [pix]")
    plt.tight_layout()
    plt.savefig(outpath, dpi=150)
    plt.close()
    
def save_frame_with_detections(
    img,
    sources,
    outpath,
    title="",
    p_low=0.1,
    p_high=99.5,
    max_points=None,
):
    vmin = np.nanpercentile(img, p_low)
    vmax = np.nanpercentile(img, p_high)
    if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin >= vmax:
        vmin, vmax = float(np.nanmin(img)), float(np.nanmax(img))

    plt.figure(figsize=(10, 8))
    ax = plt.gca()
    ax.imshow(img, origin="lower", vmin=vmin, vmax=vmax)

    if sources is not None and len(sources) > 0:
        x = np.array(sources["xcentroid"], float)
        y = np.array(sources["ycentroid"], float)

        if max_points is not None and len(x) > max_points:
            x = x[:max_points]
            y = y[:max_points]

        ax.scatter(
            x, y,
            s=25,
            facecolors="none",
            edgecolors="yellow",
            linewidths=0.8,
            alpha=0.8,
            label=f"detections ({len(x)})"
        )
        ax.legend(loc="upper right")

    ax.set_title(
        f"{title}\npercentiles: {p_low}%–{p_high}%"
    )
    ax.set_xlabel("x [pix]")
    ax.set_ylabel("y [pix]")
    plt.tight_layout()
    plt.savefig(outpath, dpi=150)
    plt.close()

def plot_fullframe(img, det_xy, tgt_xy, comp_xy, outpath, title):
    vmin, vmax = stretch_limits(img, 0.1, 99.5)
    plt.figure(figsize=(10, 8))
    ax = plt.gca()
    ax.imshow(img, vmin=vmin, vmax=vmax, origin="lower")
    ax.set_title(title)
    ax.set_xlabel("x [pix]")
    ax.set_ylabel("y [pix]")

    if OVERLAY_DETECTIONS and det_xy is not None and det_xy.shape[0] > 0:
        nshow = min(det_xy.shape[0], MAX_DETECTIONS_TO_DRAW)
        ax.scatter(det_xy[:nshow, 0], det_xy[:nshow, 1],
                   s=DETECTION_MARKER_SIZE, facecolors="none",
                   edgecolors="yellow", linewidths=0.8, alpha=0.7)

    ap_t = CircularAperture([tuple(tgt_xy)], r=AP_RADIUS)
    an_t = CircularAnnulus([tuple(tgt_xy)], r_in=ANN_IN, r_out=ANN_OUT)
    ap_t.plot(color="white", lw=1.6, alpha=0.95, axes=ax)
    an_t.plot(color="white", lw=1.0, alpha=0.6, axes=ax)
    ax.plot([tgt_xy[0]], [tgt_xy[1]], marker="+", markersize=14, markeredgewidth=2)

    if len(comp_xy) > 0:
        ap_c = CircularAperture([tuple(x) for x in comp_xy], r=AP_RADIUS)
        an_c = CircularAnnulus([tuple(x) for x in comp_xy], r_in=ANN_IN, r_out=ANN_OUT)
        ap_c.plot(color="cyan", lw=1.3, alpha=0.95, axes=ax)
        an_c.plot(color="cyan", lw=0.9, alpha=0.6, axes=ax)

    plt.tight_layout()
    plt.savefig(outpath, dpi=150)
    plt.close()


def plot_timeseries(y, outpath, title, ylabel, invert_y=False):
    plt.figure(figsize=(10, 4))
    plt.plot(y, marker=".", linestyle="-")
    plt.title(title)
    plt.xlabel("Frame index")
    plt.ylabel(ylabel)
    if invert_y:
        plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(outpath, dpi=150)
    plt.close()


# =============================================================================
# ================================= MAIN ======================================
# =============================================================================

meta = parse_meta_block(META_TEXT)
if ("TARGET_RA_DEG" not in meta) or ("TARGET_DEC_DEG" not in meta):
    raise RuntimeError("Failed to parse RA/Dec from META_TEXT.")
target_coord = SkyCoord(meta["TARGET_RA_DEG"] * u.deg, meta["TARGET_DEC_DEG"] * u.deg, frame="icrs")

t_log = meta["DATEOBS_TIME"]
if LOG_TIME_IS_MID_EXPOSURE:
    t0_start = t_log - TimeDelta(EXPOSURE_TIME / 2.0, format="sec")
    t0_mid = t_log
else:
    t0_start = t_log
    t0_mid = t_log + TimeDelta(EXPOSURE_TIME / 2.0, format="sec")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open video: {VIDEO_PATH}")

n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"[info] video: n_frames={n_frames}  shape=({H},{W})")

# choose diagnostic frames without loading cube
rng = np.random.default_rng(RANDOM_SEED)
diag_idxs = set(sorted([int(i) for i in rng.choice(np.arange(n_frames),
                                                   size=min(N_RANDOM_FRAMES, n_frames),
                                                   replace=False)]))

# --- Read frame0 ---
# --- Build stacked image for astrometry (better SNR) ---
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
print(f"[info] Building astrometry stack from first {ASTROMETRY_STACK_N} frames...")
ast_img, n_used = read_and_stack_first_frames(
    cap, n_stack=ASTROMETRY_STACK_N, method=ASTROMETRY_STACK_METHOD
)
save_frame_percentiles(
    ast_img,
    OUTDIR / f"debug_stack_percentiles_{OUTPUT_SUFFIX}.png",
    title=f"Astrometry stack (n={n_used}, method={ASTROMETRY_STACK_METHOD})",
    p_low=0.1, p_high=99.5
)
print("[out] debug:", OUTDIR / f"debug_stack_percentiles_{OUTPUT_SUFFIX}.png")
print(f"[info] Stack built with n={n_used} frames using method='{ASTROMETRY_STACK_METHOD}'")

# optional: save a cube FITS header-only note (we won't save cube itself)
if SAVE_CUBE_FITS:
    cube_path = OUTDIR / f"cube_{OUTPUT_SUFFIX}.fits"
    hdu = fits.PrimaryHDU(np.zeros((1, H, W), dtype=np.uint16))
    write_common_header(hdu.header, meta, t0_start, t0_mid)
    hdu.header["NOTE"] = "Cube not written (stream mode)."
    hdu.writeto(cube_path, overwrite=True)
    print("[out] placeholder cube:", cube_path)

print("[info] Detecting stars on stacked image...")
sources0 = get_stars_positions(ast_img)

save_frame_with_detections(
    ast_img,
    sources0,
    OUTDIR / f"debug_stack_with_detections_{OUTPUT_SUFFIX}.png",
    title=f"Astrometry stack (n={n_used})",
    p_low=0.1,
    p_high=99.5,
)
print("[out] debug stack + detections saved")

if sources0 is None or len(sources0) < MIN_MATCHES:
    raise RuntimeError("Too few stars detected on stacked image. Tune DAO params.")
print(f"[debug] stars detected on stack: {len(sources0)}")

ref_xy = np.column_stack([
    np.array(sources0["xcentroid"], float),
    np.array(sources0["ycentroid"], float)
])

print("[info] Solving astrometry locally on stacked image...")
wcs_header0, wcs0 = solve_astrometry_local(
    sources0,
    init_ra_deg=meta["TARGET_RA_DEG"],
    init_dec_deg=meta["TARGET_DEC_DEG"],
)

x0, y0 = wcs0.world_to_pixel(target_coord)
tgt0_xy = np.array([[float(x0), float(y0)]], dtype=np.float32)
print(f"[info] target pixel (stack) ~ x={x0:.2f}, y={y0:.2f}")

# pick comparison stars from the same detection table used for astrometry
comp_xy0 = pick_comp_stars_from_sources(sources0, float(x0), float(y0), (H, W))
comp0_xy = np.array(comp_xy0, dtype=np.float32)
print(f"[info] comps picked: {len(comp_xy0)}")

# IMPORTANT: reset to frame0 before starting the streaming loop
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

A_last = np.array([[1, 0, 0],
                   [0, 1, 0]], dtype=np.float32)

aper_area = np.pi * (AP_RADIUS ** 2)

# store small 1D arrays only (cheap)
f_t, f_csum, f_rel = [], [], []
bkg_t_pp, bkg_c_pp_median = [], []
dx_list, dy_list, ninliers_list = [], [], []

# CSV streaming write
with open(CSV_FILE, "w", newline="") as fcsv:
    wcsv = csv.writer(fcsv)
    wcsv.writerow([
        "frame",
        "time_start_isot", "time_mid_isot", "time_end_isot",
        "flux_target", "flux_comp_sum", "flux_rel",
        "dmag_target",
        "flux_target_norm", "flux_ref_norm", "bkg_norm",
        "bkg_target_perpix", "bkg_comp_median_perpix",
        "median_dmag_target",
        "inliers", "aff_dx", "aff_dy"
    ])

    # process frame0 + remaining frames
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    print("[info] Streaming photometry loop...")
    for i in tqdm(range(n_frames)):
        ret, frame_bgr = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY).astype(np.float32)

        # time stamps
        t_start = t0_start + TimeDelta(i * DT_FRAME, format="sec")
        t_mid   = t_start + TimeDelta(EXPOSURE_TIME / 2.0, format="sec")
        t_end   = t_start + TimeDelta(EXPOSURE_TIME, format="sec")

        det_xy = None

        # detect + estimate transform
        nin = -1
        if (i % DETECT_EVERY_N == 0) or (i == 0):
            sources_i = get_stars_positions(frame)
            if sources_i is not None and len(sources_i) >= MIN_MATCHES:
                cur_xy = np.column_stack([np.array(sources_i["xcentroid"], float),
                                          np.array(sources_i["ycentroid"], float)])
                det_xy = cur_xy.copy()

                ref_m, cur_m = match_mutual_nn_by_radius(ref_xy, cur_xy, MATCH_MAX_DIST_PX, MIN_MATCHES)
                if ref_m is not None:
                    A_new, nin2 = estimate_similarity_ransac(ref_m, cur_m, MIN_MATCHES)
                    nin = nin2
                    if A_new is not None:
                        A_last = A_new
                else:
                    nin = 0
            else:
                nin = 0

        ninliers_list.append(int(nin))
        dx_list.append(float(A_last[0, 2]))
        dy_list.append(float(A_last[1, 2]))

        # update target/comps positions
        tgt_i = apply_affine(A_last, tgt0_xy)[0]
        comp_i = apply_affine(A_last, comp0_xy)

        # photometry + bkg
        ft_arr, bkg_t_arr = aperture_flux_and_bkg(frame, [tuple(tgt_i)])
        fc_arr, bkg_c_arr = aperture_flux_and_bkg(frame, [tuple(xy) for xy in comp_i])

        ft = float(ft_arr[0])
        cs = float(np.nansum(fc_arr))
        fr = ft / cs if cs != 0 else np.nan

        f_t.append(ft)
        f_csum.append(cs)
        f_rel.append(fr)

        bkg_t_pp.append(float(bkg_t_arr[0]))
        bkg_c_pp_median.append(float(np.nanmedian(bkg_c_arr)))

        # diagnostics full frame
        if MAKE_DIAGNOSTIC_PLOTS and (i in diag_idxs):
            title = (
                f"Full frame {i}  {t_mid.isot}  "
                f"inliers={ninliers_list[-1]}  dx={A_last[0,2]:.2f} dy={A_last[1,2]:.2f}"
            )
            outp = OUTDIR / f"diag_full_{i:06d}_{OUTPUT_SUFFIX}.png"
            plot_fullframe(
                img=frame,
                det_xy=det_xy if OVERLAY_DETECTIONS else None,
                tgt_xy=tgt_i,
                comp_xy=comp_i,
                outpath=outp,
                title=title
            )

        # write CSV row now with placeholders for norms/dmag (filled later in a second pass)
        wcsv.writerow([
            i,
            t_start.isot, t_mid.isot, t_end.isot,
            ft, cs, fr,
            np.nan,   # dmag_target
            np.nan,   # flux_target_norm
            np.nan,   # flux_ref_norm
            np.nan,   # bkg_norm
            bkg_t_pp[-1],
            bkg_c_pp_median[-1],
            np.nan,   # median_dmag_target
            ninliers_list[-1],
            dx_list[-1], dy_list[-1]
        ])

cap.release()

# =============================================================================
# =============== SECOND PASS ON ARRAYS (cheap) FOR PLOTS/CSV ==================
# =============================================================================
f_t = np.array(f_t, dtype=float)
f_csum = np.array(f_csum, dtype=float)
f_rel = np.array(f_rel, dtype=float)
bkg_t_pp = np.array(bkg_t_pp, dtype=float)

good = np.isfinite(f_rel) & (f_rel > 0) & np.isfinite(f_csum) & (f_csum > 0)

ref_rel = np.nanmedian(f_rel[good]) if np.any(good) else np.nan
f_rel_norm = f_rel / ref_rel
dmag = -2.5 * np.log10(f_rel_norm)
dmag_med = float(np.nanmedian(dmag[np.isfinite(dmag)])) if np.any(np.isfinite(dmag)) else np.nan

ft_ref = np.nanmedian(f_t[np.isfinite(f_t) & (f_t > 0)])
fc_ref = np.nanmedian(f_csum[np.isfinite(f_csum) & (f_csum > 0)])
ft_norm = f_t / ft_ref
fc_norm = f_csum / fc_ref
bkg_norm = (bkg_t_pp * (np.pi * AP_RADIUS**2)) / fc_ref

# rewrite CSV with final columns filled (still memory-light; just text rewrite)
CSV_FINAL = OUTDIR / f"lightcurve_{OUTPUT_SUFFIX}_final.csv"
with open(CSV_FILE, "r", newline="") as fin, open(CSV_FINAL, "w", newline="") as fout:
    r = csv.reader(fin)
    w = csv.writer(fout)
    header = next(r)
    w.writerow(header)
    for i, row in enumerate(r):
        # row layout matches what we wrote above
        row[7]  = f"{dmag[i]:.12g}" if np.isfinite(dmag[i]) else "nan"
        row[8]  = f"{ft_norm[i]:.12g}" if np.isfinite(ft_norm[i]) else "nan"
        row[9]  = f"{fc_norm[i]:.12g}" if np.isfinite(fc_norm[i]) else "nan"
        row[10] = f"{bkg_norm[i]:.12g}" if np.isfinite(bkg_norm[i]) else "nan"
        row[13] = f"{dmag_med:.12g}" if np.isfinite(dmag_med) else "nan"
        w.writerow(row)

print("[out] CSV (final):", CSV_FINAL)

# =============================================================================
# ================================= PLOTS =====================================
# =============================================================================
plt.figure(figsize=(10, 4))
plt.plot(f_rel, marker=".", linestyle="-")
plt.xlabel("Frame index")
plt.ylabel("Differential flux (target / sum(comps))")
plt.title(f"Differential Lightcurve - {OUTPUT_SUFFIX}")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(PLOT_FLUXREL, dpi=150)
plt.close()
print("[out] plot:", PLOT_FLUXREL)

plt.figure(figsize=(10, 4))
plt.plot(dmag, marker=".", linestyle="-", label="Target dmag")
if np.isfinite(dmag_med):
    plt.axhline(dmag_med, color="black", ls="--", lw=1.2, label="Median")
plt.gca().invert_yaxis()
plt.xlabel("Frame index")
plt.ylabel("dmag [mag]")
plt.title(f"Differential Magnitude - {OUTPUT_SUFFIX}")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(PLOT_DMAG, dpi=150)
plt.close()
print("[out] plot:", PLOT_DMAG)

plt.figure(figsize=(12, 3))
plt.plot(ft_norm, marker="o", ms=2.5, lw=1.2, label="Target (norm)")
plt.plot(fc_norm, marker="o", ms=2.5, lw=1.2, label="Reference sum (norm)")
plt.plot(bkg_norm, marker="o", ms=2.5, lw=1.2, label="Background (scaled)")
plt.title("Normalized target / reference / background")
plt.xlabel("Frame index")
plt.ylabel("Normalized flux")
plt.grid(True, alpha=0.3)
plt.legend(loc="lower left")
plt.tight_layout()
plt.savefig(PLOT_NORM3, dpi=150)
plt.close()
print("[out] plot:", PLOT_NORM3)

plot_timeseries(dx_list, OUTDIR / f"aff_dx_{OUTPUT_SUFFIX}.png",
                f"Affine dx - {OUTPUT_SUFFIX}", "dx [pix]")
plot_timeseries(dy_list, OUTDIR / f"aff_dy_{OUTPUT_SUFFIX}.png",
                f"Affine dy - {OUTPUT_SUFFIX}", "dy [pix]")
plot_timeseries(ninliers_list, OUTDIR / f"aff_inliers_{OUTPUT_SUFFIX}.png",
                f"RANSAC inliers - {OUTPUT_SUFFIX}", "N inliers")

print(f"[out] Diagnostic full-frame plots saved in: {OUTDIR}" if MAKE_DIAGNOSTIC_PLOTS else "[info] Diagnostics disabled")
print("DONE.")

[info] video: n_frames=7020  shape=(1920,1080)
[info] Building astrometry stack from first 5 frames...
[out] debug: out_Uranus1/debug_stack_percentiles_Uranus1.png
[info] Stack built with n=5 frames using method='sum'
[info] Detecting stars on stacked image...
[out] debug stack + detections saved
[debug] stars detected on stack: 26
[info] Solving astrometry locally on stacked image...
[info] Preparing astrometry.net solver with 26 stars...
[info] target pixel (stack) ~ x=558.83, y=1002.20
[info] comps picked: 5
[info] Streaming photometry loop...


100%|██████████| 7020/7020 [32:38<00:00,  3.58it/s]
/var/folders/xg/8flyksh91h501f6w253syw5m0000gn/T/ipykernel_80910/34038232.py:759: RuntimeWarning: invalid value encountered in log10
  dmag = -2.5 * np.log10(f_rel_norm)


[out] CSV (final): out_Uranus1/lightcurve_Uranus1_final.csv
[out] plot: out_Uranus1/plot_fluxrel_Uranus1.png
[out] plot: out_Uranus1/plot_dmag_Uranus1.png
[out] plot: out_Uranus1/plot_norm_target_ref_bkg_Uranus1.png
[out] Diagnostic full-frame plots saved in: out_Uranus1
DONE.
